In [ ]:
# Importa as bibliotecas

import os
from dotenv import load_dotenv

import pandas as pd
import psycopg2 as pg
import sqlalchemy
from sqlalchemy import create_engine
import panel as pn

: 

In [ ]:
# Carrega as variáveis do arquivo .env

load_dotenv()

In [ ]:
# Lê as variáveis de ambiente

DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

In [ ]:
# Cria conexão com psycopg2 usando as variáveis carregadas

con = pg.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)

In [ ]:
# Define a string de conexão para o SQLAlchemy, utilizando as variáveis do .env
# Cria o objeto engine do SQLAlchemy que será usado para conectar e executar comandos no banco

cnx = f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}'

engine = sqlalchemy.create_engine(cnx)

In [ ]:
# Executa a consulta SQL para buscar todos os 
# registros da tabela 'pessoa' no banco PostgreSQL 
# e carrega o resultado em um DataFrame do pandas


query = "select * from profissionais_saude;" 
df = pd.read_sql_query(query, cnx)

df

In [ ]:
#campos de texto

#declare esta variável para usar na consulta de campos em branco
flag=''

# Cria widgets interativos para o usuário inserir ou selecionar dados:

pnome = pn.widgets.TextInput(
    name = "Primeiro Nome",
    value='',
    placeholder='Digite o nome',
    disabled=False
)

sobrenome = pn.widgets.TextInput(
    name = "Sobrenome",
    value='',
    placeholder='Digite o sobrenome',
    disabled=False
)

cpf = pn.widgets.TextInput(
    name="CPF",
    value='',
    placeholder='Digite o CPF',
    disabled=False
)

cns = pn.widgets.TextInput(
    name="CNS",
    value='',
    placeholder='Digite o CNS',
    disabled=False
)

datanasc = pn.widgets.DatePicker(
    name='Data de Nascimento',
    disabled=False
)

telefone = pn.widgets.TextInput(
    name="Telefone",
    value='',
    placeholder='Digite o telefone',
    disabled=False
)

rua = pn.widgets.TextInput(
    name="Rua",
    value='',
    placeholder='Digite a rua',
    disabled=False
)

bairro = pn.widgets.TextInput(
    name="Bairro",
    value='',
    placeholder='Digite o bairro',
    disabled=False
)

numero = pn.widgets.TextInput(
    name="Numero",
    value='',
    placeholder='Digite o numero',
    disabled=False
)

sexo = pn.widgets.RadioButtonGroup(
    name='Sexo',
    options={
        'Não Informado': 'N',
        'Masculino': 'M',
        'Feminino': 'F'
    },
    button_type='primary',  # Ou 'success', 'warning' etc
    button_style='outline',  # Estilo moderno
    width=300,
    value='N'  # Valor padrão
)

tipo_profissional = pn.widgets.RadioButtonGroup(
    name='Tipo de Profissional',
    options=['Pessoa', 'Paciente', 'Médico', 'Enfermeiro', 'Técnico Enfermagem', 'Agente Comunitário', 'Recepcionista'],
    button_type='primary',  # Ou 'success', 'warning' etc
    button_style='outline',  # Estilo moderno
    width=300,
    value = 'Pessoa'
)

# Widget específico para CRM (aparecerá apenas quando selecionar "Médico")
crm_input = pn.widgets.TextInput(name='CRM', visible=False)
microarea_input = pn.widgets.TextInput(name='Micro-Área', visible=False)
corenenfermeiro_input = pn.widgets.TextInput(name='COREN', visible=False)
corentecnico_input = pn.widgets.TextInput(name='COREN', visible=False)
recepcionista_input = pn.widgets.TextInput(name='Turno', visible=False)
priori_input = pn.widgets.Select(
    name='Grupo Prioritário',
    options={'Sim': True, 'Não': False},
    visible=False
)


def update_campos_profissionais(event):
    tipo = tipo_profissional.value
    crm_input.visible = (tipo == 'Médico')
    microarea_input.visible = (tipo == 'Agente Comunitário')
    corenenfermeiro_input.visible = (tipo == 'Enfermeiro')
    corentecnico_input.visible = (tipo == 'Técnico Enfermagem')
    priori_input.visible = (tipo == 'Paciente')
    recepcionista_input.visible = (tipo == 'Recepcionista')

tipo_profissional.param.watch(update_campos_profissionais, 'value')
update_campos_profissionais(None)


In [64]:
# Cria quatro botões para as ações principais da aplicação CRUD:
# Consultar, Inserir, Excluir e Atualizar registros no banco de dados


buttonConsultar = pn.widgets.Button(name='Consultar', button_type='default')

buttonInserir = pn.widgets.Button(name='Inserir', button_type='default')

buttonExcluir = pn.widgets.Button(name='Excluir', button_type='default')

buttonAtualizar = pn.widgets.Button(name='Atualizar', button_type='default')

In [ ]:
def queryAll():
    """
    Consulta todos os registros da tabela 'pessoa' no banco de dados e retorna
    um widget Tabulator para exibição interativa dos dados.

    Returns:
        pn.widgets.Tabulator: Widget que exibe a tabela com todos os dados da tabela 'pessoa'.
    """
    query = f"select * from pessoa"
    df = pd.read_sql_query(query, cnx)
    return pn.widgets.Tabulator(df, show_index=False)


# consultar        
# neste exemplo o método de consulta usa o dataframe do pandas como retorno. Note que a flag é usada para ignorar quando um 
# campo for null (condição é sempre verdadeira). Veja que para cpf, que é uma string, foi usado '{cpf.value})' como parametro
# e para dnr que é numérico, foi usado {dnr.value} (sem aspas simples).
def on_consultar():
    """
    Consulta registros nas tabelas pessoa / médico / ACS / enfermeiro / técnico
    filtrando pelo CPF informado.  Se o CPF estiver vazio, retorna todos.
    """
    try:
        cpf_val  = cpf.value_input or ""        # string (pode vir vazia)
        flag_val = flag                         # sua flag de controle

        # --- médico ---------------------------------------------------------
        if crm_input.visible:
            query = """
                SELECT
                    p.pnome AS nome,
                    p.sobrenome AS sobrenome,
                    ps.id_profissional,
                    p.cpf  AS cpf_pessoa,
                    p.data_nasc AS data_natalícia
                FROM medico  AS m
                JOIN pessoa  AS p ON m.cpf_medico = p.cpf
                JOIN profissionais_saude AS ps ON ps.cpf_profissional = p.cpf
                WHERE (%s = %s OR p.cpf = %s);
            """
            params = (cpf_val, flag_val, cpf_val)

        # --- agente comunitário ---------------------------------------------
        elif microarea_input.visible:
            query = """
                SELECT 
                    p.pnome AS nome,
                    p.sobrenome AS sobrenome,
                    ps.id_profissional,
                    p.cpf AS cpf_pessoa,
                    p.data_nasc,
                    ac.microarea
                FROM Agente_Comunitario AS ac
                JOIN pessoa             AS p ON ac.cpf_ac = p.cpf
                JOIN profissionais_saude AS ps ON ps.cpf_profissional = p.cpf
                WHERE (%s = %s OR p.cpf = %s);
            """
            params = (cpf_val, flag_val, cpf_val)
        elif recepcionista_input.visible:
            query = """
                SELECT 
                    p.pnome AS nome,
                    p.sobrenome AS sobrenome,
                    ps.id_profissional,
                    p.cpf AS cpf_pessoa,
                    p.data_nasc,
                    rep.turno as turno
                FROM Recepcionista AS rep
                JOIN pessoa             AS p ON rep.cpf_recepcionista = p.cpf
                JOIN profissionais_saude AS ps ON ps.cpf_profissional = p.cpf
                WHERE (%s = %s OR p.cpf = %s);
            """
            params = (cpf_val, flag_val, cpf_val)

        # --- enfermeiro ------------------------------------------------------
        elif corenenfermeiro_input.visible:
            query = """
                SELECT 
                    p.pnome AS nome,
                    p.sobrenome AS sobrenome,
                    ps.id_profissional,
                    p.cpf AS cpf_pessoa,
                    en.coren,
                    p.data_nasc
                FROM enfermeiro en
                JOIN pessoa p ON en.cpf_enfermeiro = p.cpf
                JOIN profissionais_saude AS ps ON ps.cpf_profissional = p.cpf
                WHERE (%s = %s OR p.cpf = %s);
            """
            params = (cpf_val, flag_val, cpf_val)

        # --- técnico de enfermagem ------------------------------------------
        elif corentecnico_input.visible:
            query = """
                SELECT 
                    p.pnome AS nome,
                    p.sobrenome AS sobrenome,
                    ps.id_profissional,
                    p.cpf AS cpf_pessoa,
                    tec.coren,
                    p.data_nasc
                FROM tecnico_enfermagem AS tec
                JOIN pessoa             AS p ON tec.cpf_tecnico = p.cpf
                JOIN profissionais_saude AS ps ON ps.cpf_profissional = p.cpf
                WHERE (%s = %s OR p.cpf = %s);
            """
            params = (cpf_val, flag_val, cpf_val)
        elif priori_input.visible:
            query = """
                SELECT 
                    p.pnome AS nome,
                    p.sobrenome AS sobrenome,
                    p.cpf AS cpf_pessoa,
                    p.data_nasc,
                    pa.grupoprioritario AS Grupo_Prioritario
                FROM paciente AS pa
                JOIN pessoa             AS p ON pa.cpf_paciente = p.cpf
                WHERE (%s = %s OR p.cpf = %s);
            """
            params = (cpf_val, flag_val, cpf_val)

        # --- pessoa (fallback) ----------------------------------------------
        else:
            query  = "SELECT * FROM pessoa WHERE (%s = %s OR cpf = %s);"
            params = (cpf_val, flag_val, cpf_val)

        # executa com parâmetros — ***params precisa ser tupla ou lista!***
        df = pd.read_sql_query(query, cnx, params=params)

        return pn.widgets.Tabulator(df, show_index=False)

    except Exception as e:
        return pn.pane.Alert(f"Erro ao consultar: {e}")



def on_inserir():
    """
    Insere um novo registro em 'pessoa' e, se aplicável, também em
    'profissionais_saude' e na tabela correspondente ao tipo de profissional.
    """
    try:
        # --- valores base para a tabela pessoa -----------------------------
        cpf_val       = cpf.value_input.strip()
        pnome_val     = pnome.value_input.strip()
        sobrenome_val = sobrenome.value_input.strip()
        nasc_val      = datanasc.value
        tel_val       = telefone.value_input.strip()
        cns_val       = cns.value_input.strip()
        sexo_val      = sexo.value if isinstance(sexo.value, str) else sexo.value["value"]
        rua_val       = rua.value_input.strip()
        bairro_val    = bairro.value_input.strip()
        num_val       = numero.value_input.strip()
        grupo_val     = priori_input.value


        with con:  # inicia uma transação (commit/rollback automático)
            with con.cursor() as cur:
                # 1) Inserir em pessoa
                cur.execute("""
                    INSERT INTO pessoa (
                        cpf, pnome, sobrenome, data_nasc,
                        telefone, cns, sexo, rua, bairro, numero
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    ON CONFLICT (cpf) DO NOTHING;
                """, (
                    cpf_val, pnome_val, sobrenome_val, nasc_val,
                    tel_val, cns_val, sexo_val, rua_val, bairro_val, num_val
                ))
                if tipo_profissional.value == 'Paciente':
                    cur.execute("""
                        INSERT INTO paciente (cpf_paciente, grupoprioritario)
                        VALUES (%s, %s)
                        ON CONFLICT (cpf_paciente) DO NOTHING;
                    """, (cpf_val, grupo_val))
                    
                    df = pd.read_sql_query("SELECT * FROM paciente", con)

                else:
                # 2) Inserir em profissionais_saude (necessário antes de médico, enfermeiro, técnico)
                    cur.execute("""
                        INSERT INTO profissionais_saude (cpf_profissional)
                        VALUES (%s)
                        ON CONFLICT (cpf_profissional) DO NOTHING;
                    """, (cpf_val,))

                    # 3) Inserir na tabela de tipo profissional (condicional)
                    if tipo_profissional.value == 'Médico':
                        # Médico
                        
                        cur.execute("""
                            INSERT INTO medico (crm, cpf_medico)
                            VALUES (%s, %s)
                            ON CONFLICT (crm) DO NOTHING;
                        """, (
                            crm_input.value_input.strip(),
                            cpf_val
                        ))
                        df = pd.read_sql_query("SELECT p.pnome, m.* FROM medico m join pessoa p on p.cpf = m.cpf_medico", con)
                    elif tipo_profissional.value == 'Recepcionista':
                        cur.execute("""
                            INSERT INTO recepcionista (cpf_recepcionista, turno)
                            VALUES (%s, %s)
                            ON CONFLICT (cpf_recepcionista) DO NOTHING;
                        """, (
                            cpf_val,
                            recepcionista_input.value_input.strip()
                        ))
                        df = pd.read_sql_query("SELECT p.pnome, re.* FROM Recepcionista re join pessoa p on p.cpf = re.cpf_recepcionista", con)

                    elif tipo_profissional.value == 'Agente Comunitário':
                        # Agente Comunitário
                    
                        cur.execute("""
                            INSERT INTO Agente_Comunitario (cpf_ac, microarea)
                            VALUES (%s, %s)
                            ON CONFLICT (cpf_ac) DO NOTHING;
                        """, (
                            cpf_val,
                            microarea_input.value_input.strip()
                        ))
                        df = pd.read_sql_query("SELECT p.pnome, ac.* FROM Agente_Comunitario ac join pessoa p on p.cpf = ac.cpf_ac", con)

                    elif tipo_profissional.value == 'Enfermeiro':
                        # Enfermeiro
                        cur.execute("""
                            INSERT INTO Enfermeiro (coren, cpf_enfermeiro)
                            VALUES (%s, %s)
                            ON CONFLICT (coren) DO NOTHING;
                        """, (
                            corenenfermeiro_input.value_input.strip(),
                            cpf_val
                        ))
                        df = pd.read_sql_query("SELECT p.pnome, en.* FROM enfermeiro en join pessoa p on p.cpf = en.cpf_enfermeiro", con)

                    elif tipo_profissional.value == 'Técnico Enfermagem':
                        # Técnico de Enfermagem
                        cur.execute("""
                            INSERT INTO Tecnico_Enfermagem (coren, cpf_tecnico)
                            VALUES (%s, %s)
                            ON CONFLICT (coren) DO NOTHING;
                        """, (
                            corentecnico_input.value_input.strip(),
                            cpf_val
                        ))
                        df = pd.read_sql_query("SELECT p.pnome, tec.* FROM tecnico_enfermagem tec join pessoa p on p.cpf = tec.cpf_tecnico", con)

        # --- Consulta atualizada após inserção
        
        con.commit()
        return pn.widgets.Tabulator(df)

    except Exception as e:
        return pn.pane.Alert(f"Não foi possível inserir: {str(e)}")


def on_atualizar():
    """
    Atualiza os campos nome e salario do registro identificado pelo CPF informado.

    Returns:
        pn.widgets.Tabulator ou pn.pane.Alert: Tabela atualizada ou alerta em caso de erro.
    """
    try:
        cpf_val       = cpf.value_input.strip()
        pnome_val     = pnome.value_input.strip()
        sobrenome_val = sobrenome.value_input.strip()
        nasc_val      = datanasc.value
        tel_val       = telefone.value_input.strip()
        cns_val       = cns.value_input.strip()
        sexo_val      = sexo.value if isinstance(sexo.value, str) else sexo.value["value"]
        rua_val       = rua.value_input.strip()
        bairro_val    = bairro.value_input.strip()
        num_val       = numero.value_input.strip()
        cursor= con.cursor()
        cursor.execute("UPDATE pessoa SET pnome = %s, sobrenome = %s, " \
        " data_nasc = %s, telefone = %s, cns = %s, sexo = %s, rua = %s, bairro = %s, numero = %s  WHERE cpf = %s",
           (pnome_val, sobrenome_val, nasc_val, tel_val, cns_val, sexo_val, rua_val, bairro_val, num_val, cpf_val))
        
        cursor.query
        cursor.query
        con.commit()
        return queryAll()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível atualizar: {str(e)}')
    


def on_excluir():
    """
    Exclui o registro da tabela 'pessoa' com o CPF informado.

    Returns:
        pn.widgets.Tabulator ou pn.pane.Alert: Tabela atualizada ou alerta em caso de erro.
    """
    try:
        cursor= con.cursor()
        cursor.execute("DELETE FROM medico WHERE cpf_medico = %s", (cpf.value_input,))
        cursor.execute("DELETE FROM enfermeiro WHERE cpf_enfermeiro = %s", (cpf.value_input,))
        cursor.execute("DELETE FROM tecnico_enfermagem WHERE cpf_tecnico = %s", (cpf.value_input,))
        cursor.execute("DELETE FROM agente_comunitario WHERE cpf_ac = %s", (cpf.value_input,))
        cursor.execute("DELETE FROM profissionais_saude WHERE cpf_profissional = %s", (cpf.value_input,))
        cursor.execute("DELETE FROM paciente WHERE cpf_paciente = %s", (cpf.value_input,))
        cursor.execute("delete from pessoa where cpf=%s", (cpf.value_input,))
        rows_deleted = cursor.rowcount
        con.commit()
        return queryAll()
    except Exception as e:
        cursor.execute("ROLLBACK")            
        cursor.close() 
        print(f'Não foi possível atualizar: {str(e)}')
        return pn.pane.Alert('Não foi possível excluir!')


In [ ]:
visivel = pn.Column(visible=True)  # antes de table_creator()

def table_creator(cons, ins, atu, exc):
    result = None
    if cons:
        result = on_consultar()
    if ins:
        result = on_inserir()
    if atu:
        result = on_atualizar()
    if exc:
        result = on_excluir()

    if result is not None:
        visivel[:] = [result]
        visivel.visible = True
    return result


In [ ]:
# Cria uma ligação interativa (bind) entre os botões e a função que executa a ação correspondente,
# atualizando a tabela na interface sempre que algum botão for clicado.

interactive_table = pn.bind(table_creator, buttonConsultar, buttonInserir, buttonAtualizar, buttonExcluir)

In [ ]:
# Monta o layout da interface com Panel:
# - Coluna esquerda com o título, os campos de entrada e os botões de ação
# - Coluna direita com a tabela interativa que mostra os dados do banco
# O método `.servable()` permite que essa interface seja exibida ao rodar o Panel server

visivel = pn.Column(
            interactive_table,
            styles={'background': 'white',
                    'border-radius': '10px',
                    'padding': '10px',
                    'box-shadow': '0px 0px 10px black',
                    'margin-top' : '0px',
            },
            visible = False
        )


pn.extension('tabulator', 
    raw_css=[
        '''
        body {
            background-color: #2072D0;
        }
        
        .fundo_branco{
            background-color: white;
        }
        
        .tabulator{
            border-radius: 5px;
            box-shadow: 0px 0px 5px black;
        }
        '''
    ]
)

pn.Row(
    pn.pane.Markdown(
        '**Unidade Básica de Saúde**', 
        
        styles={'font-family' : 'Trebuchet MS',
                'font-size': '64px',
                'margin' : '-75px',
                'margin-left' : '100px'}
    ),
    
    pn.layout.HSpacer(),
    pn.pane.Markdown(
      '**Tela Cadastro**',
      styles={'font-family' : 'Trebuchet MS',
                'font-size': '64px',
                'margin' : '-75px',
                'text-align' : 'center'}
    ),
    pn.layout.HSpacer(),
        
    styles = {'background-color': 'white',
              'width' : '100%',
              'height' : '75px',
              'box-shadow' : '0px 0px 15px black'}
).servable()

pn.GridBox(
    pn.Row(
        pn.Column(
            tipo_profissional,
        ),
        
        styles = {'margin-left' : '18px',
                  'margin-top' : '30px',
                  'margin-botton' : '0px',}
    ),
            
    pn.Column(
        pn.Row(
            pnome,
            sobrenome,
        ),
        pn.Row(
            cpf,
            datanasc,
        ),
        pn.Row(
            sexo,
            styles={'margin-left': 'auto',
                    'margin-right': 'auto',
                    'margin-top': '15px',
                    'margin-bottom': '15px'}
        ),
        pn.Row(
            cns,
            telefone,
        ),
        pn.Row(
            rua,
            bairro,
        ),
        pn.Row(
            numero,
            priori_input,
            crm_input,
            microarea_input,
            corenenfermeiro_input,
            corentecnico_input,
            recepcionista_input,
        ),
        styles = {'margin-left' : 'auto',
                  'margin-right' : 'auto',
                  'margin-top' : '0px'}
    ),
    

    pn.Row(
        pn.Row(buttonConsultar),
        pn.Row(buttonInserir),
        pn.Row(buttonAtualizar),
        pn.Row(buttonExcluir),

        
        styles = {'margin': 'auto',
                  'margin-top': '0px'},
    ),
    
    styles = {'background-color': 'white', 
              'margin': 'auto', 
              'width': '715px', 
              'height': '500px', 
              'margin-top': '50px',
              'border-radius': '15px',
              'box-shadow' : '0px 0px 13px black'}
).servable()

pn.Row(
    visivel,
    
    styles = {'margin': 'auto',
            'margin-top': '100px'},
).servable()